In [22]:
import pandas as pd
import numpy as np

from collections import defaultdict
from typing import List
from scipy import stats

In [2]:
df_web_logs = pd.read_csv('../data/2022-04-01T12_df_web_logs.csv')
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


## Сколько выбросов удалять
  

С одной стороны, при удалении выбросов снижается дисперсия, что приводит к увеличению чувствительности теста. С другой стороны, при удалении выбросов уменьшается размер выборки, что приводит к уменьшению чувствительности.

Сравните мощности тестов с разной долей удаляемых данных. Используйте данные о времени работы бэкенда 2022-04-01T12_df_web_logs.csv в период с 2022-03-01 по 2022-03-08. Уровень значимости — 0.05. Размеры групп — 1000 человек (размер выборок будет больше, так как на одного человека приходится много значений). Проверяем гипотезу о равенстве средних с помощью теста Стьюдента. Ожидаемый эффект — увеличение времени обработки на 1%. Эффект в синтетических А/В-тестах добавляем умножением на константу.

В ответ введите номера вариантов, упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между np.quantile(values, 0.01) и np.quantile(values, 0.99). Квантили вычислять для каждой групп отдельно.

In [31]:
def estimate_error(data_logs: pd.DataFrame,
                   begin_date: str=None,
                   end_date: str=None,
                   effect: float=1,
                   percent_remove: tuple=(0.02, 0.2,
                                        2.0, 10, 20)):

    data_period = data_logs[['user_id', 'date', 'load_time']].copy()

    if begin_date:
        data_period = data_period[data_period['date'] >= begin_date]
    if end_date:
        data_period = data_period[data_period['date'] < end_date]

    result_power = defaultdict(list)
    sample_size = 1000
    epsilon = 1 + effect / 100
    for _ in range(10000):
        users_a, users_b = np.random.choice(data_period['user_id'], (2, sample_size), False)
        origin_a = data_period.loc[data_period['user_id'].isin(users_a), 'load_time'].values
        origin_b = data_period.loc[data_period['user_id'].isin(users_b), 'load_time'].values
        origin_b *= epsilon

        for r in percent_remove:
            q = r / 100 / 2
            begin_a = np.quantile(origin_a, q)
            end_a = np.quantile(origin_a, 1 - q)
            group_a = origin_a[(origin_a > begin_a) & (origin_a < end_a)]

            begin_b = np.quantile(origin_b, q)
            end_b = np.quantile(origin_b, 1 - q)
            group_b = origin_b[(origin_b > begin_b) & (origin_b < end_b)]

            pvalue = stats.ttest_ind(group_a, group_b).pvalue
            result_power[r].append(pvalue < 0.05)

    for k, v in result_power.items():
        print(f'Outlier delete {k}%: {np.mean(v)}')

In [32]:
estimate_error(df_web_logs,
               begin_date='2022-03-01',
               end_date='2022-03-08')

Outlier delete 0.02%: 0.0801
Outlier delete 0.2%: 0.3421
Outlier delete 2.0%: 0.9616
Outlier delete 10%: 0.9769
Outlier delete 20%: 0.9829


## Сколько выбросов удалять — 2
  

Выполните то же задание, изменив способ добавления эффекта. Эффект в синтетических А/В-тестах добавляем добавлением константы к 1% данных.

В ответ введите номера вариантов упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между np.quantile(values, 0.01) и np.quantile(values, 0.99). Квантили вычислять для каждой группы отдельно.

In [38]:
def estimate_error2(data_logs: pd.DataFrame,
                   begin_date: str=None,
                   end_date: str=None,
                   effect: float=1,
                   percent_remove: tuple=(0.02, 0.2,
                                        2.0, 10, 20)):

    data_period = data_logs[['user_id', 'date', 'load_time']].copy()

    if begin_date:
        data_period = data_period[data_period['date'] >= begin_date]
    if end_date:
        data_period = data_period[data_period['date'] < end_date]

    result_power = defaultdict(list)
    sample_size = 1000
    for _ in range(10000):
        users_a, users_b = np.random.choice(data_period['user_id'], (2, sample_size), False)
        origin_a = data_period.loc[data_period['user_id'].isin(users_a), 'load_time'].values
        origin_b = data_period.loc[data_period['user_id'].isin(users_b), 'load_time'].values
        indexs = np.random.choice(np.arange(len(origin_b)), len(origin_b) // 100, False)
        origin_b[indexs] += origin_b.mean() * (effect / 100) * len(origin_b) / len(indexs)

        for r in percent_remove:
            q = r / 100 / 2
            begin_a = np.quantile(origin_a, q)
            end_a = np.quantile(origin_a, 1 - q)
            group_a = origin_a[(origin_a > begin_a) & (origin_a < end_a)]

            begin_b = np.quantile(origin_b, q)
            end_b = np.quantile(origin_b, 1 - q)
            group_b = origin_b[(origin_b > begin_b) & (origin_b < end_b)]

            pvalue = stats.ttest_ind(group_a, group_b).pvalue
            result_power[r].append(pvalue < 0.05)

    for k, v in result_power.items():
        print(f'Outlier delete {k}%: {np.mean(v)}')

In [39]:
estimate_error2(df_web_logs,
               begin_date='2022-03-01',
               end_date='2022-03-08')

Outlier delete 0.02%: 0.0877
Outlier delete 0.2%: 0.3372
Outlier delete 2.0%: 0.5146
Outlier delete 10%: 0.3218
Outlier delete 20%: 0.3427
